In [1]:
# Import main python script
from main.geolocator import (
    GeoHelper
)

# Import utility scripts
from main.maputils import (
    color_dec,
    color_hex,
    setup_map,
)

# Import geohashing library for illustration purposes
import geohash_hilbert as ghh

# Import folium for map plotting
from folium import plugins
import folium

In [2]:
### Set constants...

# Location 'A' that we are interested in determining
# whether it is inside a building or not (it is not!!!) 
lat_A = 55.39594370406802; lon_A = 10.388305276123184


# Location 'B' that we are interested in determining
# whether it is inside a building or not (it is!!!) 
lat_B = 55.396264327327984; lon_B = 10.389163305977231

# Center the map here...
MAP_CENTER = [(lat_A + lat_B) / 2, (lon_A + lon_B) / 2]

In [3]:
# Plot location 'A' and 'B' on a map and inspect their locations.
# Validate that one location is infact inside a building and 
# another is outside a building...
map_ = setup_map(
    zoom_start = 16,
    center = MAP_CENTER,
)

# A (outside a building)
c1 = "red"
folium.CircleMarker(
    location=[lat_A, lon_A],
    radius=5,
    popup="Location A",
    color=c1,
    fill=True,
    fill_color=c1,
).add_to(map_)

# B (inside a building)
c2 = "blue"
folium.CircleMarker(
    location=[lat_B, lon_B],
    radius=5,
    popup="Location A",
    color=c2,
    fill=True,
    fill_color=c2,
).add_to(map_)

# ... Also plot the bounding box encoded by the geohash that we use 
# to query and index the geospatial features in Redis.
geohash = ghh.encode(lon_A, lat_A, precision = 5, bits_per_char = 6)
obj = ghh.rectangle(geohash); bbox_rectangle = obj["geometry"]["coordinates"][0]
swapped_coords = [(v[1], v[0]) for v in bbox_rectangle]
folium.PolyLine(
    swapped_coords,
    color = "white",
    weight = 2,
    opacity = 1,
).add_to(map_)

# Finally, show the map
map_

In [4]:
# Create a 'geohelper' for running location queries 
gh = GeoHelper()

DEBUG:root:Pinging the Redis server...
DEBUG:root:The Redis server responded with True!
DEBUG:root:Client connection to localhost:6379/1 is up!


In [5]:
# Check if location A is inside a building... 
# This is the first time the query is run, so no data is cached in Redis
result_A = gh.is_inside(lat_A, lon_A, "building")
print("Is location 'A' inside a building: ", result_A)

DEBUG:root:Function: '_get_geoset'. Information: Requesting new information through the Overpass API!
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:12345
DEBUG:urllib3.connectionpool:http://localhost:12345 "GET /api/interpreter?data=%5Bout%3Ajson%5D%5Bbbox%3A55.39306640625%2C10.382080078125%2C55.3985595703125%2C10.39306640625%5D%5Btimeout%3A25%5D%3B%0A++++++++++++%28%0A++++++++++++++++node%5B%27building%27%5D%3B%0A++++++++++++++++way%5B%27building%27%5D%3B%0A++++++++++++++++relation%5B%27building%27%5D%3B%0A++++++++++++%29%3B%0A++++++++++++out+geom%3B%0A++++++++++++ HTTP/1.1" 200 None


Is location 'A' inside a building:  False


In [6]:
# Check if location B is inside a building... 
# This is the second time a query is run in the same area, so the data is cached in Redis!
result_B = gh.is_inside(lat_B, lon_B, "building")
print("Is location 'B' inside a building: ", result_B)

DEBUG:root:Function: 'get_geoset'. Information: Reading cached data from Redis!


Is location 'B' inside a building:  True


In [7]:
# Create a new folium map instance to inspect and summarize the data we worked with
map_ = setup_map(
    zoom_start = 16,
    center = MAP_CENTER,
)


# A (outside a building)
c1 = "red"
folium.CircleMarker(
    location=[lat_A, lon_A],
    radius=5,
    popup="Location A",
    color=c1,
    fill=True,
    fill_color=c1,
).add_to(map_)

# B (inside a building)
c2 = "blue"
folium.CircleMarker(
    location=[lat_B, lon_B],
    radius=5,
    popup="Location A",
    color=c2,
    fill=True,
    fill_color=c2,
).add_to(map_)


# ... Also plot the bounding box encoded by the geohash that we use to query and 
# index the geospatial features in Redis.
geohash = ghh.encode(lon_A, lat_A, precision = 5, bits_per_char = 6)
geohash_data = ghh.rectangle(geohash)
geohash_bbox = geohash_data["bbox"]
# Swap elements in bbox. We get elements (lon, lat), but we want (lat, lon)
obj = ghh.rectangle(geohash); bbox_rectangle = obj["geometry"]["coordinates"][0]
swapped_coords = [(v[1], v[0]) for v in bbox_rectangle]
folium.PolyLine(
    swapped_coords,
    color = "white",
    weight = 2,
    opacity = 1,
).add_to(map_)


# ... Also plot the polygons that were loaded into Redis and are in the vicinity
# of location 'A' and 'b'
polygons = gh.get_geoset(lat = lat_A, lon = lon_A)
for polygon in polygons:
    folium.PolyLine(
        list(polygon.exterior.coords),
        color = "pink",
        weight = 2,
        opacity = 1,
    ).add_to(map_)

# Finally, show the map
map_

DEBUG:root:Function: 'get_geoset'. Information: Reading cached data from Redis!
